In [1]:
# mount drive https://datascience.stackexchange.com/questions/29480/uploading-images-folder-from-my-system-into-google-colab
# login with your google account and type authorization code to mount on your google drive.
import os
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
pip install imutils

In [3]:
import numpy as np
import cv2
from PIL import Image
from google.colab.patches import cv2_imshow
import imutils
import math

In [4]:
# multiple cascades: https://github.com/Itseez/opencv/tree/master/data/haarcascades

face_cascade = cv2.CascadeClassifier('/gdrive/My Drive/Code/face_preprocessing/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('/gdrive/My Drive/Code/face_preprocessing/haarcascade_eye.xml')

img = cv2.imread('/gdrive/My Drive/jisu1.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.3, 3)

eye_point = []
for (x,y,w,h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
    
    eyes = eye_cascade.detectMultiScale(roi_gray)
    eyes = eyes[:2]
    for (ex,ey,ew,eh) in eyes:
        xx = ex + ew // 2
        yy = ey + eh // 2
        eye_point.append((x+xx, y+yy))
        r = (ew + eh) // 4
        cv2.circle(roi_color, (xx, yy), r, (0,0,0), 2)

cv2_imshow(img)

if len(eye_point) == 2:
    eye1 = eye_point[0]
    eye2 = eye_point[1]
    length = math.sqrt(((eye1[0] - eye2[0]) ** 2) + ((eye1[1] - eye2[1]) ** 2))

    if eye1[0] > eye2[0]:
        right = eye1
        left = eye2
    elif eye1[0] < eye2[0]:
        right = eye2
        left = eye1

    if left[1] > right[1]:
        angle = math.atan2((right[1]-left[1]), (right[0]-left[0]))
        theta = angle*(180/math.pi);
        rotated = imutils.rotate(img,theta)
    elif left[1] < right[1]:
        angle = math.atan2((left[1]-right[1]), (right[0]-left[0]))
        theta = angle*(180/math.pi);
        rotated = imutils.rotate(img,-theta)

    cropped = rotated[int(left[1] - 0.6 * length):int(left[1] + 1.8 * length), int(left[0] - 0.6 * length):int(right[0] + 0.6 * length)]
    resized = cv2.resize(cropped, dsize = (120, 140), interpolation=cv2.INTER_AREA)
    mask_region = resized[50:, :]

    cv2_imshow(rotated)
    cv2_imshow(cropped)
    cv2_imshow(mask_region)

    cv2.destroyAllWindows()